## Imports

In [1]:
from collections import Counter, defaultdict
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models.word2vec import Word2Vec
from gensim.models import Word2Vec
import gensim
import re
import numpy as np
import pylab as pl

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import learning_curve, GridSearchCV
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# imports needed and logging
import gzip
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

## Reading Data 

### Reading the BNC written data 

In [3]:
# # Natural Language Toolkit: Plaintext Corpus Reader
# #
# # Copyright (C) 2001-2019 NLTK Project
# # Author: Edward Loper <edloper@gmail.com>
# # URL: <http://nltk.org/>
# # For license information, see LICENSE.TXT

# """Corpus reader for the XML version of the British National Corpus."""

# from nltk.corpus.reader.util import concat
# from nltk.corpus.reader.xmldocs import XMLCorpusReader, XMLCorpusView, ElementTree


# class BNCCorpusReader(XMLCorpusReader):
#     """Corpus reader for the XML version of the British National Corpus.

#     For access to the complete XML data structure, use the ``xml()``
#     method.  For access to simple word lists and tagged word lists, use
#     ``words()``, ``sents()``, ``tagged_words()``, and ``tagged_sents()``.

#     You can obtain the full version of the BNC corpus at
#     http://www.ota.ox.ac.uk/desc/2554

#     If you extracted the archive to a directory called `BNC`, then you can
#     instantiate the reader as::

#         BNCCorpusReader(root='BNC/Texts/', fileids=r'[A-K]/\w*/\w*\.xml')

#     """

#     def __init__(self, root, fileids, lazy=True):
#         XMLCorpusReader.__init__(self, root, fileids)
#         self._lazy = lazy

#     def words(self, fileids=None, strip_space=True, stem=True):
#         """
#         :return: the given file(s) as a list of words
#             and punctuation symbols.
#         :rtype: list(str)

#         :param strip_space: If true, then strip trailing spaces from
#             word tokens.  Otherwise, leave the spaces on the tokens.
#         :param stem: If true, then use word stems instead of word strings.
#         """
#         return self._views(fileids, False, None, strip_space, stem)

#     def tagged_words(self, fileids=None, c5=True, strip_space=True, stem=True):
#         """
#         :return: the given file(s) as a list of tagged
#             words and punctuation symbols, encoded as tuples
#             ``(word,tag)``.
#         :rtype: list(tuple(str,str))

#         :param c5: If true, then the tags used will be the more detailed
#             c5 tags.  Otherwise, the simplified tags will be used.
#         :param strip_space: If true, then strip trailing spaces from
#             word tokens.  Otherwise, leave the spaces on the tokens.
#         :param stem: If true, then use word stems instead of word strings.
#         """
#         tag = 'c5' if c5 else 'pos'
#         return self._views(fileids, False, tag, strip_space, stem)

#     def sents(self, fileids=None, strip_space=True, stem=True):
#         """
#         :return: the given file(s) as a list of
#             sentences or utterances, each encoded as a list of word
#             strings.
#         :rtype: list(list(str))

#         :param strip_space: If true, then strip trailing spaces from
#             word tokens.  Otherwise, leave the spaces on the tokens.
#         :param stem: If true, then use word stems instead of word strings.
#         """
#         return self._views(fileids, True, None, strip_space, stem)

#     def tagged_sents(self, fileids=None, c5=True, strip_space=True, stem=True):
#         """
#         :return: the given file(s) as a list of
#             sentences, each encoded as a list of ``(word,tag)`` tuples.
#         :rtype: list(list(tuple(str,str)))

#         :param c5: If true, then the tags used will be the more detailed
#             c5 tags.  Otherwise, the simplified tags will be used.
#         :param strip_space: If true, then strip trailing spaces from
#             word tokens.  Otherwise, leave the spaces on the tokens.
#         :param stem: If true, then use word stems instead of word strings.
#         """
#         tag = 'c5' if c5 else 'pos'
#         return self._views(
#             fileids, sent=True, tag=tag, strip_space=strip_space, stem=stem
#         )

#     def _views(self, fileids=None, sent=False, tag=False, strip_space=True, stem=True):
#         """A helper function that instantiates BNCWordViews or the list of words/sentences."""
#         f = BNCWordView if self._lazy else self._words
#         return concat(
#             [
#                 f(fileid, sent, tag, strip_space, stem)
#                 for fileid in self.abspaths(fileids)
#             ]
#         )

#     def _words(self, fileid, bracket_sent, tag, strip_space, stem):
#         """
#         Helper used to implement the view methods -- returns a list of
#         words or a list of sentences, optionally tagged.

#         :param fileid: The name of the underlying file.
#         :param bracket_sent: If true, include sentence bracketing.
#         :param tag: The name of the tagset to use, or None for no tags.
#         :param strip_space: If true, strip spaces from word tokens.
#         :param stem: If true, then substitute stems for words.
#         """
#         result = []

#         xmldoc = ElementTree.parse(fileid).getroot()
#         for xmlsent in xmldoc.findall('.//s'):
#             sent = []
#             for xmlword in _all_xmlwords_in(xmlsent):
#                 word = xmlword.text
#                 if not word:
#                     word = ""  # fixes issue 337?
#                 if strip_space or stem:
#                     word = word.strip()
#                 if stem:
#                     word = xmlword.get('hw', word)
#                 if tag == 'c5':
#                     word = (word, xmlword.get('c5'))
#                 elif tag == 'pos':
#                     word = (word, xmlword.get('pos', xmlword.get('c5')))
#                 sent.append(word)
#             if bracket_sent:
#                 result.append(BNCSentence(xmlsent.attrib['n'], sent))
#             else:
#                 result.extend(sent)

#         assert None not in result
#         return result

In [4]:
# def _all_xmlwords_in(elt, result=None):
#     if result is None:
#         result = []
#     for child in elt:
#         if child.tag in ('w'):
#             result.append(child)
#         else:
#             _all_xmlwords_in(child, result)
#     return result


# class BNCSentence(list):
#     """
#     A list of words, augmented by an attribute ``num`` used to record
#     the sentence identifier (the ``n`` attribute from the XML).
#     """

#     def __init__(self, num, items):
#         self.num = num
#         list.__init__(self, items)

In [5]:
# class BNCWordView(XMLCorpusView):
#     """
#     A stream backed corpus view specialized for use with the BNC corpus.
#     """

#     tags_to_ignore = set(
#         ['stext', 'pb', 'gap', 'vocal', 'event',
#             'unclear', 'shift', 'pause', 'align', 'c']
#     )
#     """These tags are ignored. For their description refer to the
#     technical documentation, for example,
#     http://www.natcorp.ox.ac.uk/docs/URG/ref-vocal.html

#     """

#     def __init__(self, fileid, sent, tag, strip_space, stem):
#         """
#         :param fileid: The name of the underlying file.
#         :param sent: If true, include sentence bracketing.
#         :param tag: The name of the tagset to use, or None for no tags.
#         :param strip_space: If true, strip spaces from word tokens.
#         :param stem: If true, then substitute stems for words.
#         """
#         if sent:
#             tagspec = '.*/s'
#         else:
#             tagspec = '.*/s/(.*/)?(w)'
#         self._sent = sent
#         self._tag = tag
#         self._strip_space = strip_space
#         self._stem = stem

#         self.title = None  #: Title of the document.
#         self.author = None  #: Author of the document.
#         self.editor = None  #: Editor
#         self.resps = None  #: Statement of responsibility

#         XMLCorpusView.__init__(self, fileid, tagspec)

#         # Read in a tasty header.
#         self._open()
#         self.read_block(self._stream, '.*/teiHeader$', self.handle_header)
#         self.close()

#         # Reset tag context.
#         self._tag_context = {0: ()}

#     def handle_header(self, elt, context):
#         # Set up some metadata!
#         titles = elt.findall('titleStmt/title')
#         if titles:
#             self.title = '\n'.join(title.text.strip() for title in titles)

#         authors = elt.findall('titleStmt/author')
#         if authors:
#             self.author = '\n'.join(author.text.strip() for author in authors)

#         editors = elt.findall('titleStmt/editor')
#         if editors:
#             self.editor = '\n'.join(editor.text.strip() for editor in editors)

#         resps = elt.findall('titleStmt/respStmt')
#         if resps:
#             self.resps = '\n\n'.join(
#                 '\n'.join(resp_elt.text.strip() for resp_elt in resp) for resp in resps
#             )

#     def handle_elt(self, elt, context):
#         if self._sent:
#             return self.handle_sent(elt)
#         else:
#             return self.handle_word(elt)

#     def handle_word(self, elt):
#         word = elt.text
#         if not word:
#             word = ""  # fixes issue 337?
#         if self._strip_space or self._stem:
#             word = word.strip()
#         if self._stem:
#             word = elt.get('hw', word)
#         if self._tag == 'c5':
#             word = (word, elt.get('c5'))
#         elif self._tag == 'pos':
#             word = (word, elt.get('pos', elt.get('c5')))
#         return word

#     def handle_sent(self, elt):
#         sent = []
#         for child in elt:
#             if child.tag in ('mw', 'hi', 'corr', 'trunc'):
#                 sent += [self.handle_word(w) for w in child]
#             elif child.tag in ('w'):
#                 sent.append(self.handle_word(child))
#             elif child.tag not in self.tags_to_ignore:
#                 raise ValueError('Unexpected element %s' % child.tag)
#         return BNCSentence(elt.attrib['n'], sent)

In [6]:
# parser = BNCCorpusReader(root='BNC/Texts/', fileids=r'[A-K]/\w*/\w*\.xml')

### Spliting data into train/dev/test set

In [7]:
DATA_SET_PATH = "dataset/input.txt"

In [8]:
X, y = [], []
with open(DATA_SET_PATH, "r") as infile:
    for line in infile:
        text, label = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X.append(text.split())
        y.append(label)
X, y = np.array(X), np.array(y)

In [9]:
TRAIN_SET_PATH = "dataset/train_set.txt"
DEV_SET_PATH = "dataset/dev_set.txt"
TEST_SET_PATH = "dataset/test_set.txt"

In [10]:
X_train, y_train = [], []
with open(TRAIN_SET_PATH, "r") as infile:
    for line in infile:
        text, label = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X_train.append(text.split())
        y_train.append(label)
X_train, y_train = np.array(X_train), np.array(y_train)

In [11]:
X_val, y_val = [], []
with open(DEV_SET_PATH, "r") as infile:
    for line in infile:
        text, label = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X_val.append(text.split())
        y_val.append(label)
X_val, y_val = np.array(X_val), np.array(y_val)

In [12]:
X_test, y_test = [], []
with open(TEST_SET_PATH, "r") as infile:
    for line in infile:
        text, label = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X_test.append(text.split())
        y_test.append(label)
X_test, y_test = np.array(X_test), np.array(y_test)

In [13]:
for i in range(0, len(y) - 1):
    y[i] = y[i][:-1]

for i in range(0, len(y_train)):
    y_train[i] = y_train[i][:-1]
for i in range(0, len(y_train)):
    if y_train[i] == "Goal" or y_train[i] == "Cmpl":
        y_train[i] = "GC"

for i in range(0, len(y_val)):
    y_val[i] = y_val[i][:-1]
for i in range(0, len(y_val)):
    if y_val[i] == 'Goal' or y_val[i] == 'Cmpl':
        y_val[i] = 'GC'
for i in range(0, len(y_test)):
    y_test[i] = y_test[i][:-1]
for i in range(0, len(y_test)):
    if y_test[i] == 'Goal' or y_test[i] == 'Cmpl':
        y_test[i] = 'GC'

## Setting up particle features 

In [14]:
particle_features_df=pd.read_excel('particle_features.xlsx')

In [15]:
particle_features={}
for index, row in particle_features_df.iterrows():
    particle_features[particle_features_df.iloc[index]['verb']]=particle_features_df.loc[index,['about', 'along', 'around', 'back', 'by', 'down', 'in', 'off',
                          'on', 'out', 'over', 'round', 'through', 'under', 'up', 'up_0', 'up_1',
                          'up_2', 'up_3', 'up_4', 'up_5']].to_numpy(dtype='float32')
particle_features['fire']

array([0.        , 0.00512821, 0.00512821, 0.14358975, 0.        ,
       0.02051282, 0.03076923, 0.04615385, 0.15384616, 0.08205128,
       0.00512821, 0.00512821, 0.01025641, 0.        , 0.4923077 ,
       0.84375   , 0.10416666, 0.02083333, 0.        , 0.01041667,
       0.02083333], dtype=float32)

In [16]:
combined_features_V={}
for k in particle_features.keys():
    modified_key= str(k)+'_V'
    combined_features_V[modified_key]=particle_features[k]

In [17]:
combined_features_VPC={}
for k in particle_features.keys():
    modified_key= str(k)+'_VPC'
    combined_features_VPC[modified_key]=particle_features[k]

## Getting the word embeddings

### training word2vec on BNC corpus 

#### CBOW

In [18]:
X_train_list = []
# for l in d0:
for l in X_train:
    X_train_list_item = []
    for t in l:
        t = str(t)+'_VPC'
        X_train_list_item.append(t)
    X_train_list.append(X_train_list_item)
# X_val_list = []
# # for l in d0:
# for l in X_val:
#     X_val_list_item = []
#     for t in l:
#         t = str(t)+'_VPC'
#         X_val_list_item.append(t)
#     X_val_list.append(X_val_list_item)
X_val_list = []
# for l in d0:
for l in X_test:
    X_val_list_item = []
    for t in l:
        t = str(t)+'_VPC'
        X_val_list_item.append(t)
    X_val_list.append(X_val_list_item)

X_train_tagged_VPC = np.array(X_train_list)
X_val_tagged_VPC = np.array(X_val_list)

X_train_list = []
# for l in d0:
for l in X_train:
    X_train_list_item = []
    for t in l:
        t = str(t)+'_V'
        X_train_list_item.append(t)
    X_train_list.append(X_train_list_item)
# X_val_list = []
# # for l in d0:
# for l in X_val:
#     X_val_list_item = []
#     for t in l:
#         t = str(t)+'_V'
#         X_val_list_item.append(t)
#     X_val_list.append(X_val_list_item)
X_val_list = []
# for l in d0:
for l in X_test:
    X_val_list_item = []
    for t in l:
        t = str(t)+'_V'
        X_val_list_item.append(t)
    X_val_list.append(X_val_list_item)


X_train_tagged_V = np.array(X_train_list)
X_val_tagged_V = np.array(X_val_list)

In [19]:
model_BNC_sentences = Word2Vec.load("models/Trained2/model_BNC_sentences.model")
model_BNC_tagged_sentences_particle_cw = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_particle_cw.model")
model_BNC_tagged_sentences_verb_cw = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_verb_cw.model")
model_BNC_tagged_sentences_particle_up_cw = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_particle_up_cw.model")

2020-04-19 14:02:04,789 : INFO : loading Word2Vec object from models/Trained2/model_BNC_sentences.model
2020-04-19 14:02:05,209 : INFO : loading wv recursively from models/Trained2/model_BNC_sentences.model.wv.* with mmap=None
2020-04-19 14:02:05,210 : INFO : loading vectors from models/Trained2/model_BNC_sentences.model.wv.vectors.npy with mmap=None
2020-04-19 14:02:05,711 : INFO : setting ignored attribute vectors_norm to None
2020-04-19 14:02:05,712 : INFO : loading vocabulary recursively from models/Trained2/model_BNC_sentences.model.vocabulary.* with mmap=None
2020-04-19 14:02:05,713 : INFO : loading trainables recursively from models/Trained2/model_BNC_sentences.model.trainables.* with mmap=None
2020-04-19 14:02:05,715 : INFO : loading syn1neg from models/Trained2/model_BNC_sentences.model.trainables.syn1neg.npy with mmap=None
2020-04-19 14:02:06,174 : INFO : setting ignored attribute cum_table to None
2020-04-19 14:02:06,175 : INFO : loaded models/Trained2/model_BNC_sentences.mo

In [20]:
def transform_with_BNC(x):
    return np.array([
        np.mean([model_BNC_sentences[w]
                 for w in words if w in model_BNC_sentences]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC = transform_with_BNC(X_train)
# X_val_updated_BNC = transform_with_BNC(X_val)
X_val_updated_BNC = transform_with_BNC(X_test)

def transform_with_BNC_v(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_verb_cw[w]
                 for w in words if w in model_BNC_tagged_sentences_verb_cw]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC_V = transform_with_BNC_v(X_train_tagged_V)
X_val_updated_BNC_V = transform_with_BNC_v(X_val_tagged_V)

def transform_with_BNC_up(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_particle_up_cw[w]
                 for w in words if w in model_BNC_tagged_sentences_particle_up_cw]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC_VPC_up = transform_with_BNC_up(X_train_tagged_VPC)
X_val_updated_BNC_VPC_up = transform_with_BNC_up(X_val_tagged_VPC)

def transform_with_BNC_vpc(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_particle_cw[w]
                 for w in words if w in model_BNC_tagged_sentences_particle_cw]or [np.zeros(300)],
                axis=0) for words in x
    ])

X_train_updated_BNC_VPC = transform_with_BNC_vpc(X_train_tagged_VPC)
X_val_updated_BNC_VPC = transform_with_BNC_vpc(X_val_tagged_VPC)

def transform_with_particle_features(x):
    return np.array([
        np.mean([particle_features[w]
                 for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)],
                axis=0) for words in x
    ])
X_train_particle_features = transform_with_particle_features(X_train)
# X_val_particle_features = transform_with_particle_features(X_val)
X_val_particle_features = transform_with_particle_features(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be remov

In [21]:
X_train_combined_cw=np.concatenate((X_train_updated_BNC, X_train_particle_features), axis=1)
X_val_combined_cw=np.concatenate((X_val_updated_BNC, X_val_particle_features), axis=1)


X_train_combined_VPC_cw=np.concatenate((X_train_updated_BNC_VPC, X_train_particle_features), axis=1)
X_val_combined_VPC_cw=np.concatenate((X_val_updated_BNC_VPC, X_val_particle_features), axis=1)


X_train_combined_VPC_cw_up=np.concatenate((X_train_updated_BNC_VPC_up, X_train_particle_features), axis=1)
X_val_combined_VPC_cw_up=np.concatenate((X_val_updated_BNC_VPC_up, X_val_particle_features), axis=1)


X_train_combined_V_cw=np.concatenate((X_train_updated_BNC_V, X_train_particle_features), axis=1)
X_val_combined_V_cw=np.concatenate((X_val_updated_BNC_V, X_val_particle_features), axis=1)

In [22]:
X_train_combined_cw900D=np.concatenate((X_train_updated_BNC_V, X_train_updated_BNC_VPC,X_train_updated_BNC_VPC_up), axis=1)
X_val_combined_cw900D=np.concatenate((X_val_updated_BNC_V, X_val_updated_BNC_VPC,X_val_updated_BNC_VPC_up), axis=1)

In [23]:
X_train_combined_cw921D=np.concatenate((X_train_updated_BNC_V, X_train_updated_BNC_VPC,X_train_updated_BNC_VPC_up,X_train_particle_features), axis=1)
X_val_combined_cw921D=np.concatenate((X_val_updated_BNC_V, X_val_updated_BNC_VPC,X_val_updated_BNC_VPC_up,X_val_particle_features), axis=1)

In [24]:
# def transform_with_combine_features_cw(x):
#     return np.array([
#         np.concatenate(([model_BNC_sentences[w]
#                  for w in words if w in model_BNC_sentences]or [np.zeros(model_BNC_sentences['balance'].shape)],
#                        [particle_features[w]
#                  for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_cw=transform_with_combine_features_cw(X_train)
# X_val_combined_cw=transform_with_combine_features_cw(X_val)

# def transform_with_combine_features_cw_VPC(x):
#     return np.array([
#         np.concatenate((
#             [model_BNC_tagged_sentences_particle_cw[w]
#                  for w in words if w in model_BNC_tagged_sentences_particle_cw]or [np.zeros(model_BNC_tagged_sentences_particle_cw['balance_VPC'].shape)],
#                        [combined_features_VPC[w]
#                  for w in words if w in combined_features_VPC]or [np.zeros(combined_features_VPC['balance_VPC'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_VPC_cw=transform_with_combine_features_cw_VPC(X_train_tagged_VPC)
# X_val_combined_VPC_cw=transform_with_combine_features_cw_VPC(X_val_tagged_VPC)

# def transform_with_combine_features_cw_VPC_up(x):
#     return np.array([
#         np.concatenate(([model_BNC_tagged_sentences_particle_up_cw[w]
#                  for w in words if w in model_BNC_tagged_sentences_particle_up_cw]or [np.zeros(model_BNC_tagged_sentences_particle_up_cw['balance_VPC'].shape)],
#                        [combined_features_VPC[w]
#                  for w in words if w in combined_features_VPC]or [np.zeros(combined_features_VPC['balance_VPC'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_VPC_cw_up=transform_with_combine_features_cw_VPC_up(X_train_tagged_VPC)
# X_val_combined_VPC_cw_up=transform_with_combine_features_cw_VPC_up(X_val_tagged_VPC)

# def transform_with_combine_features_cw_V(x):
#     return np.array([
#         np.concatenate(([model_BNC_tagged_sentences_verb_cw[w]
#                  for w in words if w in model_BNC_tagged_sentences_verb_cw]or [np.zeros(model_BNC_tagged_sentences_verb_cw['balance_V'].shape)],
#                        [combined_features_V[w]
#                  for w in words if w in combined_features_V]or [np.zeros(combined_features_V['balance_V'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_V_cw=transform_with_combine_features_cw_V(X_train_tagged_V)
# X_val_combined_V_cw=transform_with_combine_features_cw_V(X_val_tagged_V)



In [25]:
model_BNC_sentences_sg = Word2Vec.load("models/Trained2/model_BNC_sentences_sg.model")
model_BNC_tagged_sentences_particle_sg = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_particle_sg.model")
model_BNC_tagged_sentences_particle_up_sg = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_particle_up_sg.model")
model_BNC_tagged_sentences_verb_sg = Word2Vec.load(
    "models/Trained2/model_BNC_tagged_sentences_verb_sg.model")

2020-04-19 14:02:12,455 : INFO : loading Word2Vec object from models/Trained2/model_BNC_sentences_sg.model
2020-04-19 14:02:12,928 : INFO : loading wv recursively from models/Trained2/model_BNC_sentences_sg.model.wv.* with mmap=None
2020-04-19 14:02:12,929 : INFO : loading vectors from models/Trained2/model_BNC_sentences_sg.model.wv.vectors.npy with mmap=None
2020-04-19 14:02:13,478 : INFO : setting ignored attribute vectors_norm to None
2020-04-19 14:02:13,479 : INFO : loading vocabulary recursively from models/Trained2/model_BNC_sentences_sg.model.vocabulary.* with mmap=None
2020-04-19 14:02:13,479 : INFO : loading trainables recursively from models/Trained2/model_BNC_sentences_sg.model.trainables.* with mmap=None
2020-04-19 14:02:13,480 : INFO : loading syn1neg from models/Trained2/model_BNC_sentences_sg.model.trainables.syn1neg.npy with mmap=None
2020-04-19 14:02:14,437 : INFO : setting ignored attribute cum_table to None
2020-04-19 14:02:14,438 : INFO : loaded models/Trained2/mode

In [26]:
def transform_with_BNC(x):
    return np.array([
        np.mean([model_BNC_sentences_sg[w]
                 for w in words if w in model_BNC_sentences_sg]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC = transform_with_BNC(X_train)
# X_val_updated_BNC = transform_with_BNC(X_val)
X_val_updated_BNC = transform_with_BNC(X_test)

def transform_with_BNC_v(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_verb_sg[w]
                 for w in words if w in model_BNC_tagged_sentences_verb_sg]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC_V = transform_with_BNC_v(X_train_tagged_V)
X_val_updated_BNC_V = transform_with_BNC_v(X_val_tagged_V)

def transform_with_BNC_up(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_particle_up_sg[w]
                 for w in words if w in model_BNC_tagged_sentences_particle_up_sg]or [np.zeros(300)],
                axis=0) for words in x
    ])
X_train_updated_BNC_VPC_up = transform_with_BNC_up(X_train_tagged_VPC)
X_val_updated_BNC_VPC_up = transform_with_BNC_up(X_val_tagged_VPC)

def transform_with_BNC_vpc(x):
    return np.array([
        np.mean([model_BNC_tagged_sentences_particle_sg[w]
                 for w in words if w in model_BNC_tagged_sentences_particle_sg]or [np.zeros(300)],
                axis=0) for words in x
    ])

X_train_updated_BNC_VPC = transform_with_BNC_vpc(X_train_tagged_VPC)
X_val_updated_BNC_VPC = transform_with_BNC_vpc(X_val_tagged_VPC)

def transform_with_particle_features(x):
    return np.array([
        np.mean([particle_features[w]
                 for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)],
                axis=0) for words in x
    ])
X_train_particle_features = transform_with_particle_features(X_train)
# X_val_particle_features = transform_with_particle_features(X_val)
X_val_particle_features = transform_with_particle_features(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be remov

In [27]:

X_train_combined_sg=np.concatenate((X_train_updated_BNC, X_train_particle_features), axis=1)
X_val_combined_sg=np.concatenate((X_val_updated_BNC, X_val_particle_features), axis=1)

 
X_train_combined_VPC_sg=np.concatenate((X_train_updated_BNC_VPC, X_train_particle_features), axis=1)
X_val_combined_VPC_sg=np.concatenate((X_val_updated_BNC_VPC, X_val_particle_features), axis=1)


X_train_combined_VPC_sg_up=np.concatenate((X_train_updated_BNC_VPC_up, X_train_particle_features), axis=1)
X_val_combined_VPC_sg_up=np.concatenate((X_val_updated_BNC_VPC_up, X_val_particle_features), axis=1)

X_train_combined_V_sg=np.concatenate((X_train_updated_BNC_V, X_train_particle_features), axis=1)
X_val_combined_V_sg=np.concatenate((X_val_updated_BNC_V, X_val_particle_features), axis=1)

In [28]:
X_train_combined_sg900D=np.concatenate((X_train_updated_BNC_V, X_train_updated_BNC_VPC,X_train_updated_BNC_VPC_up), axis=1)
X_val_combined_sg900D=np.concatenate((X_val_updated_BNC_V, X_val_updated_BNC_VPC,X_val_updated_BNC_VPC_up), axis=1)

In [29]:

X_train_combined_sg921D=np.concatenate((X_train_updated_BNC_V, X_train_updated_BNC_VPC,X_train_updated_BNC_VPC_up,X_train_particle_features), axis=1)
X_val_combined_sg921D=np.concatenate((X_val_updated_BNC_V, X_val_updated_BNC_VPC,X_val_updated_BNC_VPC_up,X_val_particle_features), axis=1)

#### Skip-gram

In [30]:
# X_train_list = []
# # for l in d0:
# for l in X_train:
#     X_train_list_item = []
#     for t in l:
#         t = str(t)+'_VPC'
#         X_train_list_item.append(t)
#     X_train_list.append(X_train_list_item)
# X_val_list = []
# # for l in d0:
# for l in X_val:
#     X_val_list_item = []
#     for t in l:
#         t = str(t)+'_VPC'
#         X_val_list_item.append(t)
#     X_val_list.append(X_val_list_item)


# X_train_tagged_VPC = np.array(X_train_list)
# X_val_tagged_VPC = np.array(X_val_list)

# X_train_list = []
# # for l in d0:
# for l in X_train:
#     X_train_list_item = []
#     for t in l:
#         t = str(t)+'_V'
#         X_train_list_item.append(t)
#     X_train_list.append(X_train_list_item)
# X_val_list = []
# # for l in d0:
# for l in X_val:
#     X_val_list_item = []
#     for t in l:
#         t = str(t)+'_V'
#         X_val_list_item.append(t)
#     X_val_list.append(X_val_list_item)


# X_train_tagged_V = np.array(X_train_list)
# X_val_tagged_V = np.array(X_val_list)

In [31]:
# model_BNC_sentences_sg = Word2Vec.load("model_BNC_sentences_sg.model")
# model_BNC_tagged_sentences_particle_sg = Word2Vec.load("model_BNC_tagged_sentences_particle_sg_v2.model")
# model_BNC_tagged_sentences_particle_up_sg = Word2Vec.load("model_BNC_tagged_sentences_particle_up_sg.model")
# model_BNC_tagged_sentences_verb_sg = Word2Vec.load("model_BNC_tagged_sentences_verb_sg_v2.model")

In [32]:
# def transform_with_combine_features_sg(x):
#     return np.array([
#         np.concatenate(([model_BNC_sentences_sg[w]
#                  for w in words if w in model_BNC_sentences_sg]or [np.zeros(model_BNC_sentences_sg['balance'].shape)],
#                        [particle_features[w]
#                  for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_sg=transform_with_combine_features_sg(X_train)
# X_val_combined_sg=transform_with_combine_features_sg(X_val)

# def transform_with_combine_features_sg_VPC(x):
#     return np.array([
#         np.concatenate(([model_BNC_tagged_sentences_particle_sg[w]
#                  for w in words if w in model_BNC_tagged_sentences_particle_sg]or [np.zeros(model_BNC_tagged_sentences_particle_sg['balance_VPC'].shape)],
#                        [combined_features_VPC[w]
#                  for w in words if w in combined_features_VPC]or [np.zeros(combined_features_VPC['balance_VPC'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_VPC_sg=transform_with_combine_features_sg_VPC(X_train_tagged_VPC)
# X_val_combined_VPC_sg=transform_with_combine_features_sg_VPC(X_val_tagged_VPC)

# def transform_with_combine_features_sg_VPC_up(x):
#     return np.array([
#         np.concatenate(([model_BNC_tagged_sentences_particle_up_sg[w]
#                  for w in words if w in model_BNC_tagged_sentences_particle_up_sg]or [np.zeros(model_BNC_tagged_sentences_particle_up_sg['balance_VPC'].shape)],
#                        [combined_features_VPC[w]
#                  for w in words if w in combined_features_VPC]or [np.zeros(combined_features_VPC['balance_VPC'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_VPC_sg_up=transform_with_combine_features_sg_VPC_up(X_train_tagged_VPC)
# X_val_combined_VPC_sg_up=transform_with_combine_features_sg_VPC_up(X_val_tagged_VPC)

# def transform_with_combine_features_sg_V(x):
#     return np.array([
#         np.concatenate(([model_BNC_tagged_sentences_verb_sg[w]
#                  for w in words if w in model_BNC_tagged_sentences_verb_sg]or [np.zeros(model_BNC_tagged_sentences_verb_sg['balance_V'].shape)],
#                        [combined_features_V[w]
#                  for w in words if w in combined_features_V]or [np.zeros(combined_features_V['balance_V'].shape)]),
#                 axis=None) for words in x
#     ]) 
# X_train_combined_V_sg=transform_with_combine_features_sg_V(X_train_tagged_V)
# X_val_combined_V_sg=transform_with_combine_features_sg_V(X_val_tagged_V)

### google (300D) pre-trained word embedding 

In [33]:
model_google = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True)

2020-04-19 14:02:21,740 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2020-04-19 14:03:40,714 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin


### glove pre-trained word embedding

In [34]:
GLOVE_6B_50D_PATH = "glove.6B.50d.txt"
GLOVE_840B_300D_PATH = "glove.840B.300d.txt"
encoding = "utf-8"

In [35]:
import numpy as np
with open(GLOVE_6B_50D_PATH, "rb") as lines:
    wvec = {
        line.split()[0].decode(encoding): np.array(line.split()[1:],
                                                   dtype=np.float32)
        for line in lines
    }

In [36]:
# reading glove files, this may take a while
# we're reading line by line and only saving vectors
# that correspond to words from our training set
# if you wan't to play around with the vectors and have
# enough RAM - remove the 'if' line and load everything

import struct

glove_small = {}
all_words = set(w for words in X for w in words)
with open(GLOVE_6B_50D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if (word in all_words):
            nums = np.array(parts[1:], dtype=np.float32)
            glove_small[word] = nums

glove_big = {}
with open(GLOVE_840B_300D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if word in all_words:
            nums = np.array(parts[1:], dtype=np.float32)
            glove_big[word] = nums

## Classification

In [37]:
my_tags = ['GC', 'Refl', 'Vert']

### Training with BNC word embedding (CBOW)

#### General form

##### Linear SVM

In [38]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [39]:
y_pred = clf.predict(X_val_combined_cw)

In [40]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  45.0


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.42      0.46      0.44        24
        Refl       0.11      0.33      0.17         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.45        60
   macro avg       0.38      0.42      0.37        60
weighted avg       0.50      0.45      0.47        60



##### Weighted Linear SVM

In [42]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [43]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_cw)

In [44]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  45.0


In [45]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.42      0.46      0.44        24
        Refl       0.11      0.33      0.17         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.45        60
   macro avg       0.38      0.42      0.37        60
weighted avg       0.50      0.45      0.47        60



##### SVC with 'rbf' kernel

In [46]:
X = X_train_combined_cw
Y = y_train

In [47]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [48]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [49]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 512.0, 'gamma': 0.0001220703125} with a score of 0.45


In [50]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=512.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001220703125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [51]:
y_pred = clf.predict(X_val_combined_cw)

In [52]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 45.0)


In [53]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.42      0.46      0.44        24
        Refl       0.11      0.33      0.17         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.45        60
   macro avg       0.38      0.42      0.37        60
weighted avg       0.50      0.45      0.47        60



##### Weighted SVC with 'rbf' kernel 

In [54]:
X = X_train_combined_cw
Y = y_train

In [55]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [56]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [57]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 512.0, 'gamma': 0.0001220703125} with a score of 0.45


In [58]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [59]:
y_pred = clf.predict(X_val_combined_cw)

In [60]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 55.00000000000001)


In [61]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.00      0.00      0.00        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      1.00      0.71        33

    accuracy                           0.55        60
   macro avg       0.18      0.33      0.24        60
weighted avg       0.30      0.55      0.39        60



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Logistice regression

In [62]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_cw, y_train)
y_pred = logreg.predict(X_val_combined_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.42      0.42      0.42        24
        Refl       0.00      0.00      0.00         3
        Vert       0.58      0.42      0.49        33

    accuracy                           0.40        60
   macro avg       0.33      0.28      0.30        60
weighted avg       0.49      0.40      0.44        60



##### Weighted logistice regression

In [63]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_cw, y_train)
y_pred = logreg.predict(X_val_combined_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4166666666666667
              precision    recall  f1-score   support

          GC       0.44      0.46      0.45        24
        Refl       0.00      0.00      0.00         3
        Vert       0.58      0.42      0.49        33

    accuracy                           0.42        60
   macro avg       0.34      0.29      0.31        60
weighted avg       0.50      0.42      0.45        60



#### _V

##### Linear SVM

In [64]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_V_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [65]:
y_pred = clf.predict(X_val_combined_V_cw)

In [66]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.33      0.33      0.33        24
        Refl       0.00      0.00      0.00         3
        Vert       0.52      0.42      0.47        33

    accuracy                           0.37        60
   macro avg       0.28      0.25      0.27        60
weighted avg       0.42      0.37      0.39        60



##### Weighted Linear SVM

In [68]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_V_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [69]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_V_cw)

In [70]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [71]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.33      0.33      0.33        24
        Refl       0.00      0.00      0.00         3
        Vert       0.52      0.42      0.47        33

    accuracy                           0.37        60
   macro avg       0.28      0.25      0.27        60
weighted avg       0.42      0.37      0.39        60



##### SVC with 'rbf' kernel

In [72]:
X = X_train_combined_V_cw
Y = y_train

In [73]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [74]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [75]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [76]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.0078125} with a score of 0.50


In [77]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [78]:
y_pred = clf.predict(X_val_combined_V_cw)

In [79]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 48.333333333333336)


In [80]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.44      0.29      0.35        24
        Refl       0.20      0.67      0.31         3
        Vert       0.59      0.61      0.60        33

    accuracy                           0.48        60
   macro avg       0.41      0.52      0.42        60
weighted avg       0.51      0.48      0.48        60



##### Weighted SVC with 'rbf' kernel 

In [81]:
X = X_train_combined_V_cw
Y = y_train

In [82]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [83]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [84]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [85]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [86]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.0078125} with a score of 0.50


In [87]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [88]:
y_pred = clf.predict(X_val_combined_V_cw)

In [89]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 50.0)


In [90]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.33      0.08      0.13        24
        Refl       0.33      0.67      0.44         3
        Vert       0.54      0.79      0.64        33

    accuracy                           0.50        60
   macro avg       0.40      0.51      0.41        60
weighted avg       0.45      0.50      0.43        60



##### Logistice regression

In [91]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_V_cw, y_train)
y_pred = logreg.predict(X_val_combined_V_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.38333333333333336
              precision    recall  f1-score   support

          GC       0.33      0.25      0.29        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.52      0.52        33

    accuracy                           0.38        60
   macro avg       0.29      0.26      0.27        60
weighted avg       0.43      0.38      0.40        60



##### Weighted logistice regression

In [92]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_V_cw, y_train)
y_pred = logreg.predict(X_val_combined_V_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.35      0.29      0.32        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.52      0.52        33

    accuracy                           0.40        60
   macro avg       0.29      0.27      0.28        60
weighted avg       0.43      0.40      0.41        60



#### _VPC

##### Linear SVM

In [93]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_VPC_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [94]:
y_pred = clf.predict(X_val_combined_VPC_cw)

In [95]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [96]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.43      0.38      0.40        24
        Refl       0.10      0.67      0.17         3
        Vert       0.61      0.33      0.43        33

    accuracy                           0.37        60
   macro avg       0.38      0.46      0.33        60
weighted avg       0.51      0.37      0.41        60



##### Weighted Linear SVM

In [97]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_VPC_cw, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [98]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_VPC_cw)

In [99]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [100]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.43      0.38      0.40        24
        Refl       0.10      0.67      0.17         3
        Vert       0.61      0.33      0.43        33

    accuracy                           0.37        60
   macro avg       0.38      0.46      0.33        60
weighted avg       0.51      0.37      0.41        60



##### SVC with 'rbf' kernel

In [101]:
X = X_train_combined_VPC_cw
Y = y_train

In [102]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [103]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [104]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [105]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [106]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.57


In [107]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [108]:
y_pred = clf.predict(X_val_combined_VPC_cw)

In [109]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 50.0)


In [110]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.60      0.38      0.46        24
        Refl       0.12      0.67      0.21         3
        Vert       0.66      0.58      0.61        33

    accuracy                           0.50        60
   macro avg       0.46      0.54      0.43        60
weighted avg       0.61      0.50      0.53        60



##### Weighted SVC with 'rbf' kernel 

In [111]:
X = X_train_combined_VPC_cw
Y = y_train

In [112]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [113]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [114]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [115]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [116]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.57


In [117]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [118]:
y_pred = clf.predict(X_val_combined_VPC_cw)

In [119]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 46.666666666666664)


In [120]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.54      0.29      0.38        24
        Refl       0.07      0.33      0.12         3
        Vert       0.61      0.61      0.61        33

    accuracy                           0.47        60
   macro avg       0.41      0.41      0.37        60
weighted avg       0.55      0.47      0.49        60



##### Logistice regression

In [121]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_VPC_cw, y_train)
y_pred = logreg.predict(X_val_combined_VPC_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4166666666666667
              precision    recall  f1-score   support

          GC       0.50      0.42      0.45        24
        Refl       0.00      0.00      0.00         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.42        60
   macro avg       0.37      0.29      0.32        60
weighted avg       0.53      0.42      0.47        60



##### Weighted logistice regression

In [122]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_VPC_cw, y_train)
y_pred = logreg.predict(X_val_combined_VPC_cw)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4166666666666667
              precision    recall  f1-score   support

          GC       0.48      0.50      0.49        24
        Refl       0.00      0.00      0.00         3
        Vert       0.62      0.39      0.48        33

    accuracy                           0.42        60
   macro avg       0.37      0.30      0.32        60
weighted avg       0.53      0.42      0.46        60



#### _VPC (up)

##### Linear SVM

In [123]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_VPC_cw_up, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [124]:
y_pred = clf.predict(X_val_combined_VPC_cw_up)

In [125]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [126]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.48      0.54      0.51        24
        Refl       0.08      0.33      0.13         3
        Vert       0.67      0.42      0.52        33

    accuracy                           0.47        60
   macro avg       0.41      0.43      0.39        60
weighted avg       0.56      0.47      0.50        60



##### Weighted Linear SVM

In [127]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_VPC_cw_up, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [128]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_VPC_cw_up)

In [129]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  45.0


In [130]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.50      0.48        24
        Refl       0.09      0.33      0.14         3
        Vert       0.61      0.42      0.50        33

    accuracy                           0.45        60
   macro avg       0.39      0.42      0.37        60
weighted avg       0.52      0.45      0.47        60



##### SVC with 'rbf' kernel

In [131]:
X = X_train_combined_VPC_cw_up
Y = y_train

In [132]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [133]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [134]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [135]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [136]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 0.125} with a score of 0.60


In [137]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [138]:
y_pred = clf.predict(X_val_combined_VPC_cw_up)

In [139]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 46.666666666666664)


In [140]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.44      0.33      0.38        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.61      0.60        33

    accuracy                           0.47        60
   macro avg       0.34      0.31      0.33        60
weighted avg       0.50      0.47      0.48        60



##### Weighted SVC with 'rbf' kernel 

In [141]:
X = X_train_combined_VPC_cw_up
Y = y_train

In [142]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [143]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [144]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [145]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [146]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 0.125} with a score of 0.60


In [147]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [148]:
y_pred = clf.predict(X_val_combined_VPC_cw_up)

In [149]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 50.0)


In [150]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.50      0.42      0.45        24
        Refl       0.10      0.33      0.15         3
        Vert       0.63      0.58      0.60        33

    accuracy                           0.50        60
   macro avg       0.41      0.44      0.40        60
weighted avg       0.55      0.50      0.52        60



##### Logistice regression

In [151]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_VPC_cw_up, y_train)
y_pred = logreg.predict(X_val_combined_VPC_cw_up)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.36666666666666664
              precision    recall  f1-score   support

          GC       0.46      0.54      0.50        24
        Refl       0.05      0.33      0.09         3
        Vert       0.67      0.24      0.36        33

    accuracy                           0.37        60
   macro avg       0.39      0.37      0.31        60
weighted avg       0.55      0.37      0.40        60



##### Weighted logistice regression

In [152]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_VPC_cw_up, y_train)
y_pred = logreg.predict(X_val_combined_VPC_cw_up)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.3333333333333333
              precision    recall  f1-score   support

          GC       0.42      0.46      0.44        24
        Refl       0.05      0.33      0.09         3
        Vert       0.57      0.24      0.34        33

    accuracy                           0.33        60
   macro avg       0.35      0.34      0.29        60
weighted avg       0.49      0.33      0.37        60



#### 900D combined

##### Linear SVM

In [153]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_cw900D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [154]:
y_pred = clf.predict(X_val_combined_cw900D)

In [155]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  40.0


In [156]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.36      0.38      0.37        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.45      0.49        33

    accuracy                           0.40        60
   macro avg       0.30      0.28      0.29        60
weighted avg       0.44      0.40      0.42        60



##### Weighted Linear SVM

In [157]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_cw900D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [158]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_cw900D)

In [159]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  40.0


In [160]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.36      0.38      0.37        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.45      0.49        33

    accuracy                           0.40        60
   macro avg       0.30      0.28      0.29        60
weighted avg       0.44      0.40      0.42        60



##### SVC with 'rbf' kernel

In [161]:
X = X_train_combined_cw900D
Y = y_train

In [162]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [163]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [164]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [165]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [166]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.0078125} with a score of 0.57


In [167]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [168]:
y_pred = clf.predict(X_val_combined_cw900D)

In [169]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 51.66666666666667)


In [170]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.50      0.25      0.33        24
        Refl       0.22      0.67      0.33         3
        Vert       0.59      0.70      0.64        33

    accuracy                           0.52        60
   macro avg       0.44      0.54      0.44        60
weighted avg       0.54      0.52      0.50        60



##### Weighted SVC with 'rbf' kernel 

In [171]:
X = X_train_combined_cw900D
Y = y_train

In [172]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [173]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [174]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [175]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [176]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.0078125} with a score of 0.57


In [177]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [178]:
y_pred = clf.predict(X_val_combined_cw900D)

In [179]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 50.0)


In [180]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.33      0.08      0.13        24
        Refl       0.20      0.33      0.25         3
        Vert       0.55      0.82      0.66        33

    accuracy                           0.50        60
   macro avg       0.36      0.41      0.35        60
weighted avg       0.45      0.50      0.43        60



##### Logistice regression

In [181]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_cw900D, y_train)
y_pred = logreg.predict(X_val_combined_cw900D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4666666666666667
              precision    recall  f1-score   support

          GC       0.40      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.61      0.59        33

    accuracy                           0.47        60
   macro avg       0.32      0.31      0.32        60
weighted avg       0.47      0.47      0.47        60



##### Weighted logistice regression

In [182]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_cw900D, y_train)
y_pred = logreg.predict(X_val_combined_cw900D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.38      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.48      0.51        33

    accuracy                           0.40        60
   macro avg       0.30      0.27      0.29        60
weighted avg       0.45      0.40      0.42        60



#### 921D combined

##### Linear SVM

In [183]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_cw921D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [184]:
y_pred = clf.predict(X_val_combined_cw921D)

In [185]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  40.0


In [186]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.36      0.38      0.37        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.45      0.49        33

    accuracy                           0.40        60
   macro avg       0.30      0.28      0.29        60
weighted avg       0.44      0.40      0.42        60



##### Weighted Linear SVM

In [187]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_cw921D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [188]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_cw921D)

In [189]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  40.0


In [190]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.36      0.38      0.37        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.45      0.49        33

    accuracy                           0.40        60
   macro avg       0.30      0.28      0.29        60
weighted avg       0.44      0.40      0.42        60



##### SVC with 'rbf' kernel

In [191]:
X = X_train_combined_cw921D
Y = y_train

In [192]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [193]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [194]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [195]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [196]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.0078125} with a score of 0.57


In [197]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [198]:
y_pred = clf.predict(X_val_combined_cw921D)

In [199]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 51.66666666666667)


In [200]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.50      0.25      0.33        24
        Refl       0.22      0.67      0.33         3
        Vert       0.59      0.70      0.64        33

    accuracy                           0.52        60
   macro avg       0.44      0.54      0.44        60
weighted avg       0.54      0.52      0.50        60



##### Weighted SVC with 'rbf' kernel 

In [201]:
X = X_train_combined_cw921D
Y = y_train

In [202]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [203]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [204]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [205]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [206]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.0078125} with a score of 0.57


In [207]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [208]:
y_pred = clf.predict(X_val_combined_cw921D)

In [209]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 51.66666666666667)


In [210]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.40      0.08      0.14        24
        Refl       0.33      0.67      0.44         3
        Vert       0.55      0.82      0.66        33

    accuracy                           0.52        60
   macro avg       0.43      0.52      0.41        60
weighted avg       0.48      0.52      0.44        60



##### Logistice regression

In [211]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_cw921D, y_train)
y_pred = logreg.predict(X_val_combined_cw921D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.45
              precision    recall  f1-score   support

          GC       0.40      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.56      0.58      0.57        33

    accuracy                           0.45        60
   macro avg       0.32      0.30      0.31        60
weighted avg       0.47      0.45      0.46        60



##### Weighted logistice regression

In [212]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_cw921D, y_train)
y_pred = logreg.predict(X_val_combined_cw921D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.38      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.48      0.51        33

    accuracy                           0.40        60
   macro avg       0.30      0.27      0.29        60
weighted avg       0.45      0.40      0.42        60



### Training with BNC word embedding (SkipGram)

#### General form 

##### Linear SVM

In [213]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [214]:
y_pred = clf.predict(X_val_combined_sg)

In [215]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  43.333333333333336


In [216]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.54      0.50        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.39      0.46        33

    accuracy                           0.43        60
   macro avg       0.34      0.31      0.32        60
weighted avg       0.48      0.43      0.45        60



##### Weighted Linear SVM

In [217]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [218]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_sg)

In [219]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  43.333333333333336


In [220]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.54      0.50        24
        Refl       0.00      0.00      0.00         3
        Vert       0.54      0.39      0.46        33

    accuracy                           0.43        60
   macro avg       0.34      0.31      0.32        60
weighted avg       0.48      0.43      0.45        60



##### SVC with 'rbf' kernel

In [221]:
X = X_train_combined_sg
Y = y_train

In [222]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [223]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [224]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [225]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [226]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.48


In [227]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [228]:
y_pred = clf.predict(X_val_combined_sg)

In [229]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 50.0)


In [230]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.40      0.25      0.31        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.73      0.65        33

    accuracy                           0.50        60
   macro avg       0.33      0.33      0.32        60
weighted avg       0.48      0.50      0.48        60



##### Weighted SVC with 'rbf' kernel 

In [231]:
X = X_train_combined_sg
Y = y_train

In [232]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [233]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [234]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [235]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [236]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.48


In [237]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [238]:
y_pred = clf.predict(X_val_combined_sg)

In [239]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 46.666666666666664)


In [240]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.44      0.50      0.47        24
        Refl       0.00      0.00      0.00         3
        Vert       0.62      0.48      0.54        33

    accuracy                           0.47        60
   macro avg       0.35      0.33      0.34        60
weighted avg       0.52      0.47      0.49        60



##### Logistice regression

In [241]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_sg, y_train)
y_pred = logreg.predict(X_val_combined_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.43333333333333335
              precision    recall  f1-score   support

          GC       0.46      0.46      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.43        60
   macro avg       0.35      0.30      0.33        60
weighted avg       0.51      0.43      0.47        60



##### Weighted logistice regression

In [242]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_sg, y_train)
y_pred = logreg.predict(X_val_combined_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.45
              precision    recall  f1-score   support

          GC       0.50      0.46      0.48        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.48      0.53        33

    accuracy                           0.45        60
   macro avg       0.36      0.31      0.34        60
weighted avg       0.53      0.45      0.48        60



#### _V

##### Linear SVM

In [243]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_V_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [244]:
y_pred = clf.predict(X_val_combined_V_sg)

In [245]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [246]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.29      0.21      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.50      0.52      0.51        33

    accuracy                           0.37        60
   macro avg       0.26      0.24      0.25        60
weighted avg       0.39      0.37      0.38        60



##### Weighted Linear SVM

In [247]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_V_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [248]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_V_sg)

In [249]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  36.666666666666664


In [250]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.29      0.21      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.50      0.52      0.51        33

    accuracy                           0.37        60
   macro avg       0.26      0.24      0.25        60
weighted avg       0.39      0.37      0.38        60



##### SVC with 'rbf' kernel

In [251]:
X = X_train_combined_V_sg
Y = y_train

In [252]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [253]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [254]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [255]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [256]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.47


In [257]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [258]:
y_pred = clf.predict(X_val_combined_V_sg)

In [259]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 46.666666666666664)


In [260]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.40      0.17      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.73      0.64        33

    accuracy                           0.47        60
   macro avg       0.32      0.30      0.29        60
weighted avg       0.47      0.47      0.45        60



##### Weighted SVC with 'rbf' kernel 

In [261]:
X = X_train_combined_V_sg
Y = y_train

In [262]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [263]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [264]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [265]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [266]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.47


In [267]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [268]:
y_pred = clf.predict(X_val_combined_V_sg)

In [269]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 43.333333333333336)


In [270]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.40      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.56      0.55      0.55        33

    accuracy                           0.43        60
   macro avg       0.32      0.29      0.31        60
weighted avg       0.47      0.43      0.45        60



##### Logistice regression

In [271]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_V_sg, y_train)
y_pred = logreg.predict(X_val_combined_V_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4166666666666667
              precision    recall  f1-score   support

          GC       0.40      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.52      0.54        33

    accuracy                           0.42        60
   macro avg       0.32      0.28      0.30        60
weighted avg       0.47      0.42      0.44        60



##### Weighted logistice regression

In [272]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_V_sg, y_train)
y_pred = logreg.predict(X_val_combined_V_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4166666666666667
              precision    recall  f1-score   support

          GC       0.40      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.52      0.54        33

    accuracy                           0.42        60
   macro avg       0.32      0.28      0.30        60
weighted avg       0.47      0.42      0.44        60



#### _VPC

##### Linear SVM

In [273]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_VPC_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [274]:
y_pred = clf.predict(X_val_combined_VPC_sg)

In [275]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [276]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.52      0.50      0.51        24
        Refl       0.07      0.33      0.12         3
        Vert       0.65      0.45      0.54        33

    accuracy                           0.47        60
   macro avg       0.42      0.43      0.39        60
weighted avg       0.57      0.47      0.50        60



##### Weighted Linear SVM

In [277]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_VPC_sg, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [278]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_VPC_sg)

In [279]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [280]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.52      0.50      0.51        24
        Refl       0.07      0.33      0.12         3
        Vert       0.65      0.45      0.54        33

    accuracy                           0.47        60
   macro avg       0.42      0.43      0.39        60
weighted avg       0.57      0.47      0.50        60



##### SVC with 'rbf' kernel

In [281]:
X = X_train_combined_VPC_sg
Y = y_train

In [282]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [283]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [284]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [285]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [286]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.5} with a score of 0.48


In [287]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [288]:
y_pred = clf.predict(X_val_combined_VPC_sg)

In [289]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 48.333333333333336)


In [290]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.33      0.04      0.07        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.85      0.65        33

    accuracy                           0.48        60
   macro avg       0.29      0.30      0.24        60
weighted avg       0.42      0.48      0.39        60



##### Weighted SVC with 'rbf' kernel 

In [291]:
X = X_train_combined_VPC_sg
Y = y_train

In [292]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [293]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [294]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [295]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [296]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.5} with a score of 0.48


In [297]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [298]:
y_pred = clf.predict(X_val_combined_VPC_sg)

In [299]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 45.0)


In [300]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.48      0.42      0.44        24
        Refl       0.00      0.00      0.00         3
        Vert       0.61      0.52      0.56        33

    accuracy                           0.45        60
   macro avg       0.36      0.31      0.33        60
weighted avg       0.52      0.45      0.48        60



##### Logistice regression

In [301]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_VPC_sg, y_train)
y_pred = logreg.predict(X_val_combined_VPC_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4666666666666667
              precision    recall  f1-score   support

          GC       0.50      0.50      0.50        24
        Refl       0.00      0.00      0.00         3
        Vert       0.64      0.48      0.55        33

    accuracy                           0.47        60
   macro avg       0.38      0.33      0.35        60
weighted avg       0.55      0.47      0.50        60



##### Weighted logistice regression

In [302]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_VPC_sg, y_train)
y_pred = logreg.predict(X_val_combined_VPC_sg)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4666666666666667
              precision    recall  f1-score   support

          GC       0.50      0.50      0.50        24
        Refl       0.00      0.00      0.00         3
        Vert       0.64      0.48      0.55        33

    accuracy                           0.47        60
   macro avg       0.38      0.33      0.35        60
weighted avg       0.55      0.47      0.50        60



#### _VPC (up)

##### Linear SVM

In [303]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_VPC_sg_up, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [304]:
y_pred = clf.predict(X_val_combined_VPC_sg_up)

In [305]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [306]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.50      0.54      0.52        24
        Refl       0.09      0.33      0.14         3
        Vert       0.61      0.42      0.50        33

    accuracy                           0.47        60
   macro avg       0.40      0.43      0.39        60
weighted avg       0.54      0.47      0.49        60



##### Weighted Linear SVM

In [307]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_VPC_sg_up, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [308]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_VPC_sg_up)

In [309]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [310]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.52      0.54      0.53        24
        Refl       0.08      0.33      0.13         3
        Vert       0.61      0.42      0.50        33

    accuracy                           0.47        60
   macro avg       0.40      0.43      0.39        60
weighted avg       0.55      0.47      0.49        60



##### SVC with 'rbf' kernel

In [311]:
X = X_train_combined_VPC_sg_up
Y = y_train

In [312]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [313]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [314]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [315]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [316]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.5} with a score of 0.58


In [317]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [318]:
y_pred = clf.predict(X_val_combined_VPC_sg_up)

In [319]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 55.00000000000001)


In [320]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.57      0.33      0.42        24
        Refl       0.25      0.33      0.29         3
        Vert       0.57      0.73      0.64        33

    accuracy                           0.55        60
   macro avg       0.46      0.46      0.45        60
weighted avg       0.56      0.55      0.53        60



##### Weighted SVC with 'rbf' kernel 

In [321]:
X = X_train_combined_VPC_sg_up
Y = y_train

In [322]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [323]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [324]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [325]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [326]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.5} with a score of 0.58


In [327]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [328]:
y_pred = clf.predict(X_val_combined_VPC_sg_up)

In [329]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 46.666666666666664)


In [330]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.48      0.50      0.49        24
        Refl       0.10      0.33      0.15         3
        Vert       0.60      0.45      0.52        33

    accuracy                           0.47        60
   macro avg       0.39      0.43      0.39        60
weighted avg       0.53      0.47      0.49        60



##### Logistice regression

In [331]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_VPC_sg_up, y_train)
y_pred = logreg.predict(X_val_combined_VPC_sg_up)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.5
              precision    recall  f1-score   support

          GC       0.54      0.58      0.56        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      0.48      0.52        33

    accuracy                           0.50        60
   macro avg       0.36      0.36      0.36        60
weighted avg       0.52      0.50      0.51        60



##### Weighted logistice regression

In [332]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_VPC_sg_up, y_train)
y_pred = logreg.predict(X_val_combined_VPC_sg_up)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.5
              precision    recall  f1-score   support

          GC       0.52      0.54      0.53        24
        Refl       0.17      0.33      0.22         3
        Vert       0.55      0.48      0.52        33

    accuracy                           0.50        60
   macro avg       0.41      0.45      0.42        60
weighted avg       0.52      0.50      0.51        60



#### 900D combined

##### Linear SVM

In [333]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_sg900D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [334]:
len(X_val_combined_sg900D[0])

900

In [335]:
y_pred = clf.predict(X_val_combined_sg900D)

In [336]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [337]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.46      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.47        60
   macro avg       0.35      0.32      0.34        60
weighted avg       0.51      0.47      0.48        60



##### Weighted Linear SVM

In [338]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_sg900D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [339]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_sg900D)

In [340]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [341]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.46      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.47        60
   macro avg       0.35      0.32      0.34        60
weighted avg       0.51      0.47      0.48        60



##### SVC with 'rbf' kernel

In [342]:
X = X_train_combined_sg900D
Y = y_train

In [343]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [344]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [345]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [346]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [347]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.0078125} with a score of 0.53


In [348]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [349]:
y_pred = clf.predict(X_val_combined_sg900D)

In [350]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 43.333333333333336)


In [351]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.41      0.38      0.39        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.52      0.54        33

    accuracy                           0.43        60
   macro avg       0.33      0.30      0.31        60
weighted avg       0.48      0.43      0.45        60



##### Weighted SVC with 'rbf' kernel 

In [352]:
X = X_train_combined_sg900D
Y = y_train

In [353]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [354]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [355]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [356]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [357]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.0078125} with a score of 0.53


In [358]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [359]:
y_pred = clf.predict(X_val_combined_sg900D)

In [360]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 43.333333333333336)


In [361]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.41      0.38      0.39        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      0.52      0.53        33

    accuracy                           0.43        60
   macro avg       0.32      0.30      0.31        60
weighted avg       0.47      0.43      0.45        60



##### Logistice regression

In [362]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_sg900D, y_train)
y_pred = logreg.predict(X_val_combined_sg900D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.45
              precision    recall  f1-score   support

          GC       0.43      0.42      0.43        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.45        60
   macro avg       0.34      0.31      0.32        60
weighted avg       0.50      0.45      0.47        60



##### Weighted logistice regression

In [363]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_sg900D, y_train)
y_pred = logreg.predict(X_val_combined_sg900D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.43333333333333335
              precision    recall  f1-score   support

          GC       0.41      0.38      0.39        24
        Refl       0.00      0.00      0.00         3
        Vert       0.57      0.52      0.54        33

    accuracy                           0.43        60
   macro avg       0.33      0.30      0.31        60
weighted avg       0.48      0.43      0.45        60



#### 921D combined

##### Linear SVM

In [364]:
clf = SVC(kernel='linear')
clf.fit(X_train_combined_sg921D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [365]:
y_pred = clf.predict(X_val_combined_sg921D)

In [366]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [367]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.46      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.47        60
   macro avg       0.35      0.32      0.34        60
weighted avg       0.51      0.47      0.48        60



##### Weighted Linear SVM

In [368]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_combined_sg921D, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [369]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_combined_sg921D)

In [370]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  46.666666666666664


In [371]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.46      0.46      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.47        60
   macro avg       0.35      0.32      0.34        60
weighted avg       0.51      0.47      0.48        60



##### SVC with 'rbf' kernel

In [372]:
X = X_train_combined_sg921D
Y = y_train

In [373]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [374]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [375]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [376]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [377]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 2.0, 'gamma': 0.03125} with a score of 0.50


In [378]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=2.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [379]:
y_pred = clf.predict(X_val_combined_sg921D)

In [380]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 41.66666666666667)


In [381]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.38      0.33      0.36        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.52      0.52        33

    accuracy                           0.42        60
   macro avg       0.30      0.28      0.29        60
weighted avg       0.44      0.42      0.43        60



##### Weighted SVC with 'rbf' kernel 

In [382]:
X = X_val_combined_sg921D
Y = y_train

In [383]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [384]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [385]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [386]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.00048828125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [387]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 0.00048828125} with a score of 0.45


In [388]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [389]:
y_pred = clf.predict(X_val_combined_sg921D)

In [390]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 40.0)


In [391]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.43      0.38      0.40        24
        Refl       0.07      0.33      0.11         3
        Vert       0.58      0.42      0.49        33

    accuracy                           0.40        60
   macro avg       0.36      0.38      0.33        60
weighted avg       0.50      0.40      0.44        60



##### Logistice regression

In [392]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_combined_sg921D, y_train)
y_pred = logreg.predict(X_val_combined_sg921D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.45
              precision    recall  f1-score   support

          GC       0.43      0.42      0.43        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.45        60
   macro avg       0.34      0.31      0.32        60
weighted avg       0.50      0.45      0.47        60



##### Weighted logistice regression

In [393]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_combined_sg921D, y_train)
y_pred = logreg.predict(X_val_combined_sg921D)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.45
              precision    recall  f1-score   support

          GC       0.43      0.42      0.43        24
        Refl       0.00      0.00      0.00         3
        Vert       0.59      0.52      0.55        33

    accuracy                           0.45        60
   macro avg       0.34      0.31      0.32        60
weighted avg       0.50      0.45      0.47        60



### training with google pre-trained word embedding 

In [394]:
# def transform_with_google(x):
#     return np.array([
#         np.mean([model_google[w]
#                  for w in words if w in model_google] or [np.zeros(model_google['balance'].shape)],
#                 axis=0) for words in x
#     ])
def transform_with_google(x):
    return np.array([
        np.concatenate(([model_google[w]
                 for w in words if w in model_google]or [np.zeros(model_google['balance'].shape)],
                       [particle_features[w]
                 for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)]),
                axis=None) for words in x
    ]) 
# X_train_combined_cw=transform_with_combine_features_cw(X_train)
# X_val_combined_cw=transform_with_combine_features_cw(X_val)

In [395]:
X_train_updated_google = transform_with_google(X_train)
# X_val_updated_google = transform_with_google(X_val)
X_val_updated_google = transform_with_google(X_test)

#### Linear SVM

In [396]:
clf = SVC(kernel='linear')
clf.fit(X_train_updated_google, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [397]:
y_pred = clf.predict(X_val_updated_google)

In [398]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  33.33333333333333


In [399]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.28      0.29      0.29        24
        Refl       0.00      0.00      0.00         3
        Vert       0.48      0.39      0.43        33

    accuracy                           0.33        60
   macro avg       0.25      0.23      0.24        60
weighted avg       0.38      0.33      0.35        60



#### Weighted Linear SVM

In [400]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_updated_google, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [401]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_updated_google)

In [402]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  33.33333333333333


In [403]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.28      0.29      0.29        24
        Refl       0.00      0.00      0.00         3
        Vert       0.48      0.39      0.43        33

    accuracy                           0.33        60
   macro avg       0.25      0.23      0.24        60
weighted avg       0.38      0.33      0.35        60



#### SVC with 'rbf' kernel

In [404]:
X = X_train_updated_google
Y = y_train

In [405]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [406]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [407]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [408]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [409]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.03125} with a score of 0.52


In [410]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [411]:
y_pred = clf.predict(X_val_updated_google)

In [412]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 35.0)


In [413]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.31      0.33      0.32        24
        Refl       0.00      0.00      0.00         3
        Vert       0.48      0.39      0.43        33

    accuracy                           0.35        60
   macro avg       0.26      0.24      0.25        60
weighted avg       0.39      0.35      0.37        60



#### Weighted SVC with 'rbf' kernel 

In [414]:
X = X_train_updated_google
Y = y_train

In [415]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [416]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [417]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [418]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=8.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.03125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [419]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 8.0, 'gamma': 0.03125} with a score of 0.52


In [420]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [421]:
y_pred = clf.predict(X_val_updated_google)

In [422]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 36.666666666666664)


In [423]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.32      0.33      0.33        24
        Refl       0.00      0.00      0.00         3
        Vert       0.50      0.42      0.46        33

    accuracy                           0.37        60
   macro avg       0.27      0.25      0.26        60
weighted avg       0.40      0.37      0.38        60



#### Logistice regression

In [424]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_updated_google, y_train)
y_pred = logreg.predict(X_val_updated_google)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.36666666666666664
              precision    recall  f1-score   support

          GC       0.33      0.29      0.31        24
        Refl       0.00      0.00      0.00         3
        Vert       0.50      0.45      0.48        33

    accuracy                           0.37        60
   macro avg       0.28      0.25      0.26        60
weighted avg       0.41      0.37      0.39        60



#### Weighted logistice regression

In [425]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_updated_google, y_train)
y_pred = logreg.predict(X_val_updated_google)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.36666666666666664
              precision    recall  f1-score   support

          GC       0.33      0.29      0.31        24
        Refl       0.00      0.00      0.00         3
        Vert       0.48      0.45      0.47        33

    accuracy                           0.37        60
   macro avg       0.27      0.25      0.26        60
weighted avg       0.40      0.37      0.38        60



### training with glove_small (50D) pre-trained word embedding 

In [426]:
def transform_with_glove_small(x):
    return np.array([
        np.mean([glove_small[w]
                 for w in words if w in glove_small] or [np.zeros(glove_small['balance'].shape)],
                axis=0) for words in x
    ])

In [427]:
X_train_updated_glove_small = transform_with_glove_small(X_train)
# X_val_updated_glove_small = transform_with_glove_small(X_val)
X_val_updated_glove_small = transform_with_glove_small(X_test)

#### Linear SVM

In [428]:
clf = SVC(kernel='linear')
clf.fit(X_train_updated_glove_small, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [429]:
y_pred = clf.predict(X_val_updated_glove_small)

In [430]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  38.333333333333336


In [431]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.45      0.54      0.49        24
        Refl       0.00      0.00      0.00         3
        Vert       0.56      0.30      0.39        33

    accuracy                           0.38        60
   macro avg       0.33      0.28      0.29        60
weighted avg       0.48      0.38      0.41        60



#### Weighted Linear SVM

In [432]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_updated_glove_small, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [433]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_updated_glove_small)

In [434]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  38.333333333333336


In [435]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.45      0.54      0.49        24
        Refl       0.00      0.00      0.00         3
        Vert       0.56      0.30      0.39        33

    accuracy                           0.38        60
   macro avg       0.33      0.28      0.29        60
weighted avg       0.48      0.38      0.41        60



#### SVC with 'rbf' kernel

In [436]:
X = X_train_updated_glove_small
Y = y_train

In [437]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [438]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [439]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=128.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [440]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=128.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [441]:
y_pred = clf.predict(X_val_updated_glove_small)

In [442]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 38.333333333333336)


In [443]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.41      0.50      0.45        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      0.33      0.42        33

    accuracy                           0.38        60
   macro avg       0.32      0.28      0.29        60
weighted avg       0.47      0.38      0.41        60



#### Weighted SVC with 'rbf' kernel 

In [444]:
X = X_train_updated_glove_small
Y = y_train

In [445]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_glove_small)

In [446]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [447]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [448]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=128.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [449]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 128.0, 'gamma': 0.001953125} with a score of 0.45


In [450]:
clf = SVC(C=512.0, class_weight='balanced', gamma=0.00048828125, kernel='rbf')
clf.fit(X, Y)

SVC(C=512.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.00048828125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [451]:
y_pred = clf.predict(X_val_updated_glove_small)

In [452]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 36.666666666666664)


In [453]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.43      0.50      0.46        24
        Refl       0.00      0.00      0.00         3
        Vert       0.53      0.30      0.38        33

    accuracy                           0.37        60
   macro avg       0.32      0.27      0.28        60
weighted avg       0.46      0.37      0.40        60



#### Logistice regression

In [454]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_updated_glove_small, y_train)
y_pred = logreg.predict(X_val_updated_glove_small)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.43333333333333335
              precision    recall  f1-score   support

          GC       0.44      0.50      0.47        24
        Refl       0.10      0.33      0.15         3
        Vert       0.57      0.39      0.46        33

    accuracy                           0.43        60
   macro avg       0.37      0.41      0.36        60
weighted avg       0.49      0.43      0.45        60



#### Weighted logistice regression

In [455]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_updated_glove_small, y_train)
y_pred = logreg.predict(X_val_updated_glove_small)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.41      0.46      0.43        24
        Refl       0.09      0.33      0.14         3
        Vert       0.55      0.36      0.44        33

    accuracy                           0.40        60
   macro avg       0.35      0.39      0.34        60
weighted avg       0.47      0.40      0.42        60



### training with glove_big (300D) pre-trained word embedding 

In [456]:
# def transform_with_glove_big(x):
#     return np.array([
#         np.mean([glove_big[w]
#                  for w in words if w in glove_big] or [np.zeros(glove_big['balance'].shape)],
#                 axis=0) for words in x
#     ])
def transform_with_glove_big(x):
    return np.array([
        np.concatenate(([glove_big[w]
                 for w in words if w in glove_big]or [np.zeros(glove_big['balance'].shape)],
                       [particle_features[w]
                 for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)]),
                axis=None) for words in x
    ]) 

In [457]:
X_train_updated_glove_big = transform_with_glove_big(X_train)
# X_val_updated_glove_big = transform_with_glove_big(X_val)
X_val_updated_glove_big = transform_with_glove_big(X_test)

#### Linear SVM

In [458]:
clf = SVC(kernel='linear')
clf.fit(X_train_updated_glove_big, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [459]:
y_pred = clf.predict(X_val_updated_glove_big)

In [460]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  26.666666666666668


In [461]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.24      0.25      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.37      0.30      0.33        33

    accuracy                           0.27        60
   macro avg       0.20      0.18      0.19        60
weighted avg       0.30      0.27      0.28        60



#### Weighted Linear SVM

In [462]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_updated_glove_big, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [463]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_updated_glove_big)

In [464]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  26.666666666666668


In [465]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.24      0.25      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.37      0.30      0.33        33

    accuracy                           0.27        60
   macro avg       0.20      0.18      0.19        60
weighted avg       0.30      0.27      0.28        60



#### SVC with 'rbf' kernel

In [466]:
X = X_train_updated_glove_big
Y = y_train

In [467]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [468]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [469]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [470]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [471]:
y_pred = clf.predict(X_val_updated_glove_big)

In [472]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 33.33333333333333)


In [473]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.27      0.25      0.26        24
        Refl       0.00      0.00      0.00         3
        Vert       0.45      0.42      0.44        33

    accuracy                           0.33        60
   macro avg       0.24      0.22      0.23        60
weighted avg       0.36      0.33      0.34        60



#### Weighted SVC with 'rbf' kernel 

In [474]:
X = X_train_updated_glove_big
Y = y_train

In [475]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X = X_train_updated_glove_big)

In [476]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [477]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [478]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001953125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [479]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 0.001953125} with a score of 0.48


In [480]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.001953125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.001953125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [481]:
y_pred = clf.predict(X_val_updated_glove_big)

In [482]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 35.0)


In [483]:
y_val

array(['GC', 'GC', 'GC', 'Vert', 'Vert', 'Vert', 'GC', 'GC', 'Vert',
       'Vert', 'GC', 'Vert', 'GC', 'GC', 'Vert', 'GC', 'Vert', 'Vert',
       'GC', 'Vert', 'Refl', 'Vert', 'Vert', 'Vert', 'Vert', 'Vert',
       'Vert', 'Vert', 'Vert', 'Vert', 'Vert', 'GC', 'GC', 'Vert', 'GC',
       'GC', 'Vert', 'GC', 'Vert', 'Vert', 'GC', 'Refl', 'Vert', 'GC',
       'Refl', 'Vert', 'GC', 'GC', 'GC', 'Vert', 'Vert', 'Vert', 'GC',
       'Vert', 'Vert', 'GC', 'Vert', 'Vert', 'GC', 'GC'], dtype='<U5')

In [484]:
y_pred

array(['Vert', 'Vert', 'GC', 'Vert', 'Refl', 'Vert', 'Vert', 'GC', 'Vert',
       'GC', 'Vert', 'Refl', 'Vert', 'Vert', 'Vert', 'Refl', 'GC', 'GC',
       'Vert', 'GC', 'GC', 'GC', 'GC', 'GC', 'Refl', 'Refl', 'Vert',
       'Vert', 'Vert', 'GC', 'GC', 'GC', 'Refl', 'GC', 'Vert', 'GC',
       'Vert', 'Vert', 'Vert', 'Vert', 'Vert', 'Vert', 'GC', 'Vert',
       'Vert', 'GC', 'Refl', 'Vert', 'GC', 'Vert', 'Vert', 'Vert', 'GC',
       'GC', 'Vert', 'Vert', 'Vert', 'GC', 'Vert', 'Vert'], dtype='<U5')

In [485]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.29      0.25      0.27        24
        Refl       0.00      0.00      0.00         3
        Vert       0.47      0.45      0.46        33

    accuracy                           0.35        60
   macro avg       0.25      0.23      0.24        60
weighted avg       0.37      0.35      0.36        60



#### Logistice regression

In [486]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_updated_glove_big, y_train)
y_pred = logreg.predict(X_val_updated_glove_big)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.3
              precision    recall  f1-score   support

          GC       0.29      0.21      0.24        24
        Refl       0.00      0.00      0.00         3
        Vert       0.42      0.39      0.41        33

    accuracy                           0.30        60
   macro avg       0.24      0.20      0.22        60
weighted avg       0.35      0.30      0.32        60



#### Weighted logistice regression

In [487]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_updated_glove_big, y_train)
y_pred = logreg.predict(X_val_updated_glove_big)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.2833333333333333
              precision    recall  f1-score   support

          GC       0.25      0.17      0.20        24
        Refl       0.00      0.00      0.00         3
        Vert       0.41      0.39      0.40        33

    accuracy                           0.28        60
   macro avg       0.22      0.19      0.20        60
weighted avg       0.32      0.28      0.30        60



### Particle features 

In [488]:
def transform_with_particle_features(x):
    return np.array([
        np.mean([particle_features[w]
                 for w in words if w in particle_features]or [np.zeros(particle_features['balance'].shape)],
                axis=0) for words in x
    ])


In [489]:
combined={}
for k in particle_features.keys():
    modified_key= str(k)+'_V'
    combined[modified_key]=particle_features[k]

In [490]:
def transform_with_combine_features(x):
    return np.array([
        np.concatenate(([model_BNC_tagged_sentences_verb_sg[w]
                 for w in words if w in model_BNC_tagged_sentences_verb_sg]or [np.zeros(model_BNC_tagged_sentences_verb_sg['balance_V'].shape)],
                       [combined[w]
                 for w in words if w in combined]or [np.zeros(combined['balance_V'].shape)]),
                axis=None) for words in x
    ]) 
X_train_combined=transform_with_combine_features(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [491]:
X_train_particle_features = transform_with_particle_features(X_train)
X_val_particle_features = transform_with_particle_features(X_val)

##### Linear SVM

In [492]:
clf = SVC(kernel='linear')
clf.fit(X_train_particle_features, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [493]:
y_pred = clf.predict(X_val_particle_features)

In [494]:
print("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100)

SVM Accuracy Score ->  53.333333333333336


In [495]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.45      0.21      0.29        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      0.82      0.66        33

    accuracy                           0.53        60
   macro avg       0.34      0.34      0.31        60
weighted avg       0.48      0.53      0.48        60



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Weighted Linear SVM

In [496]:
clf_balanced_class_weight = SVC(kernel='linear', class_weight='balanced')
clf_balanced_class_weight.fit(X_train_particle_features, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [497]:
y_pred_balanced_class_weight = clf_balanced_class_weight.predict(
    X_val_particle_features)

In [498]:
print("SVM Accuracy Score -> ",
      accuracy_score(y_pred_balanced_class_weight, y_val) * 100)

SVM Accuracy Score ->  35.0


In [499]:
from sklearn.metrics import classification_report
print(
    classification_report(y_val,
                          y_pred_balanced_class_weight,
                          target_names=my_tags))

              precision    recall  f1-score   support

          GC       0.50      0.33      0.40        24
        Refl       0.07      0.67      0.13         3
        Vert       0.65      0.33      0.44        33

    accuracy                           0.35        60
   macro avg       0.41      0.44      0.32        60
weighted avg       0.56      0.35      0.41        60



##### SVC with 'rbf' kernel

In [500]:
X = X_train_particle_features
Y = y_train

In [501]:
X_train_particle_features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.47540984, 0.        , 0.03278688, ..., 0.        , 0.        ,
        0.        ],
       [0.00479233, 0.        , 0.        , ..., 0.00906344, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03209243, 0.01558775, 0.05226481, ..., 0.03063063, 0.01801802,
        0.01081081],
       [0.01234568, 0.03703704, 0.03703704, ..., 0.01724138, 0.01724138,
        0.        ]], dtype=float32)

In [502]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [503]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [504]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [505]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=8.0, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [506]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 8.0} with a score of 0.47


In [507]:
clf = grid.best_estimator_
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=8.0, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [508]:
y_pred = clf.predict(X_val_particle_features)

In [509]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 48.333333333333336)


In [510]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.47      0.58      0.52        24
        Refl       0.10      0.33      0.15         3
        Vert       0.70      0.42      0.53        33

    accuracy                           0.48        60
   macro avg       0.42      0.45      0.40        60
weighted avg       0.58      0.48      0.51        60



##### Weighted SVC with 'rbf' kernel 

In [511]:
X = X_train_particle_features
Y = y_train

In [512]:
# It is usually a good idea to scale the data for SVM training.
# We are cheating a bit in this example in scaling all of the data,
# instead of fitting the transformation on the trainingset and
# just applying it on the test set.

# scaler = StandardScaler()
# X = scaler.fit_transform(X_train_updated_google)

In [513]:
C_range = 2. ** np.arange(-5, 18, 2)
gamma_range = 2. ** np.arange(-17, 4, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid,
                    cv=StratifiedKFold(n_splits=10))
grid.fit(X, Y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs...
       3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
       3.27680e+04, 1.31072e+05]),
                         'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
       1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
       5.00000000e-01, 2.00000000e+00, 8.00000000e+00])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=

In [514]:
param_grid

{'gamma': array([7.62939453e-06, 3.05175781e-05, 1.22070312e-04, 4.88281250e-04,
        1.95312500e-03, 7.81250000e-03, 3.12500000e-02, 1.25000000e-01,
        5.00000000e-01, 2.00000000e+00, 8.00000000e+00]),
 'C': array([3.12500e-02, 1.25000e-01, 5.00000e-01, 2.00000e+00, 8.00000e+00,
        3.20000e+01, 1.28000e+02, 5.12000e+02, 2.04800e+03, 8.19200e+03,
        3.27680e+04, 1.31072e+05])}

In [515]:
print("The best classifier is: ", grid.best_estimator_)

The best classifier is:  SVC(C=32.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=8.0, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [516]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 32.0, 'gamma': 8.0} with a score of 0.47


In [517]:
clf = SVC(C=32.0, class_weight='balanced', gamma=0.03125, kernel='rbf')
clf.fit(X, Y)

SVC(C=32.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.03125,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [518]:
y_pred = clf.predict(X_val_particle_features)

In [519]:
print(("SVM Accuracy Score -> ", accuracy_score(y_pred, y_val) * 100))

('SVM Accuracy Score -> ', 36.666666666666664)


In [520]:
y_val

array(['GC', 'GC', 'GC', 'Vert', 'Vert', 'Vert', 'GC', 'GC', 'Vert',
       'Vert', 'GC', 'Vert', 'GC', 'GC', 'Vert', 'GC', 'Vert', 'Vert',
       'GC', 'Vert', 'Refl', 'Vert', 'Vert', 'Vert', 'Vert', 'Vert',
       'Vert', 'Vert', 'Vert', 'Vert', 'Vert', 'GC', 'GC', 'Vert', 'GC',
       'GC', 'Vert', 'GC', 'Vert', 'Vert', 'GC', 'Refl', 'Vert', 'GC',
       'Refl', 'Vert', 'GC', 'GC', 'GC', 'Vert', 'Vert', 'Vert', 'GC',
       'Vert', 'Vert', 'GC', 'Vert', 'Vert', 'GC', 'GC'], dtype='<U5')

In [521]:
y_pred

array(['GC', 'GC', 'Vert', 'Refl', 'GC', 'Refl', 'Vert', 'Vert', 'GC',
       'GC', 'Refl', 'Refl', 'Vert', 'Refl', 'Vert', 'Refl', 'Refl',
       'Vert', 'Refl', 'Refl', 'Refl', 'Vert', 'GC', 'Vert', 'Refl',
       'Refl', 'Refl', 'Vert', 'Vert', 'Vert', 'Vert', 'Refl', 'Refl',
       'Vert', 'Refl', 'GC', 'Refl', 'GC', 'GC', 'Vert', 'Refl', 'Refl',
       'GC', 'GC', 'Vert', 'GC', 'Vert', 'Refl', 'GC', 'GC', 'Refl',
       'Vert', 'GC', 'Refl', 'Vert', 'Vert', 'Refl', 'GC', 'GC', 'Refl'],
      dtype='<U5')

In [522]:
from sklearn.metrics import classification_report
print((classification_report(y_val, y_pred, target_names=my_tags)))

              precision    recall  f1-score   support

          GC       0.47      0.33      0.39        24
        Refl       0.08      0.67      0.15         3
        Vert       0.63      0.36      0.46        33

    accuracy                           0.37        60
   macro avg       0.40      0.45      0.33        60
weighted avg       0.54      0.37      0.42        60



##### Logistice regression

In [523]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_particle_features, y_train)
y_pred = logreg.predict(X_val_particle_features)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.4
              precision    recall  f1-score   support

          GC       0.46      0.25      0.32        24
        Refl       0.00      0.00      0.00         3
        Vert       0.60      0.55      0.57        33

    accuracy                           0.40        60
   macro avg       0.35      0.27      0.30        60
weighted avg       0.51      0.40      0.44        60



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


##### Weighted logistice regression

In [524]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')
logreg = logreg.fit(X_train_particle_features, y_train)
y_pred = logreg.predict(X_val_particle_features)
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

accuracy 0.35
              precision    recall  f1-score   support

          GC       0.38      0.21      0.27        24
        Refl       0.00      0.00      0.00         3
        Vert       0.55      0.48      0.52        33

    accuracy                           0.35        60
   macro avg       0.31      0.23      0.26        60
weighted avg       0.46      0.35      0.39        60



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
